In [1]:
!pip install -q -U kaggle_environments

In [2]:
import numpy as np
import pandas as pd

import random

import matplotlib.pyplot as plt

import seaborn as sns

import math

from utils import get_score

from kaggle_environments import make, evaluate



### 0 Бот статист для оценки результата

In [3]:
%%writefile HW_bots/statistical.py

import random
from utils import get_score

action_histogram = {}


def statistical(observation, configuration):
    global action_histogram
    if observation.step == 0:
        action_histogram = {}
        return
    action = observation.lastOpponentAction
    if action not in action_histogram:
        action_histogram[action] = 0
    action_histogram[action] += 1
    mode_action = None
    mode_action_count = None
    for k, v in action_histogram.items():
        if mode_action_count is None or v > mode_action_count:
            mode_action = k
            mode_action_count = v
            continue

    return (mode_action + 1) % configuration.signs


Overwriting HW_bots/statistical.py


### 1. Агент, всегда возвращающий камень

In [4]:
#%%writefile HW_bots/rock_bot.py

def rock_bot(observation, configuration):
    return 0 

#### Игра

In [5]:
evaluate(
    "rps", #environment to use - no need to change
    ["HW_bots/rock_bot.py", "HW_bots/statistical.py"], #agents to evaluate
    configuration={"episodeSteps": 100} #number of episodes 
)

[[-98.0, 98.0]]

### 2. Агент, всегда возвращающий бумагу

In [6]:
#%%writefile HW_bots/paper_bot.py
def paper_bot(observation, configuration):
    return 1 

#### Игра со статистом

In [7]:
evaluate(
    "rps", #environment to use - no need to change
    ["HW_bots/rock_bot.py", "HW_bots/statistical.py"], #agents to evaluate
    configuration={"episodeSteps": 100} #number of episodes 
)

[[-98.0, 98.0]]

### 3. Агент, всегда возвращающий ножницы

In [8]:
#%%writefile HW_bots/scissors_bot.py

def scissors_bot(observation, configuration):
    return 2 

#### Игра со статистом

In [9]:
evaluate(
    "rps", #environment to use - no need to change
    ["HW_bots/scissors_bot.py", "HW_bots/statistical.py"], #agents to evaluate
    configuration={"episodeSteps": 100} #number of episodes 
)

[[-99.0, 99.0]]

### 4. Агент, повторящий действие соперника из прошлого хода

In [10]:
%%writefile HW_bots/copy_opponent_bot.py

import random

def copy_opponent_bot(observation, configuration):
    if observation.step>0:
        return observation.lastOpponentAction
    else:
        return random.randrange(0,configuration.signs)

   

Overwriting HW_bots/copy_opponent_bot.py


#### Игра со статистом

In [11]:
print(evaluate(
    "rps", #environment to use - no need to change
    ["HW_bots/copy_opponent_bot.py", "HW_bots/statistical.py"], #agents to evaluate
    configuration={"episodeSteps": 100} #number of episodes 
))

[[-20.0, 20.0]]


### 5. Агент, всегда использующий рандомное действие, вне зависимости от исхода раунда

In [12]:
#%%writefile HW_bots/random_bot.py
import math
from utils import get_score
import random

def random_bot(observation, configuration):
    return random.randrange(0,configuration.signs)

#### Игра со статистом

In [13]:
evaluate(
    "rps", #environment to use - no need to change
    ["HW_bots/random_bot.py", "HW_bots/statistical.py"], #agents to evaluate
    configuration={"episodeSteps": 100} #number of episodes 
)

[[0, 0]]

### 6. Агент,  последовательно пребирающий заложенные варинты.

In [14]:
%%writefile HW_bots/sort_bot.py


lastAction=None  

def sort_bot(observation, configuration):
    global lastAction
    
    if lastAction==None:
        lastAction=0
    else: 
        lastAction=(lastAction+1)%configuration.signs
    
    return lastAction

     
        

Overwriting HW_bots/sort_bot.py


#### Игра со статистом

In [15]:
evaluate(
    "rps", #environment to use - no need to change
    ["HW_bots/sort_bot.py", "HW_bots/statistical.py"], #agents to evaluate
    configuration={"episodeSteps": 100} #number of episodes 
)

[[0, 0]]

### 7. Агент,  +1 к предыдущему ходу соперника, вне зависимости от результата раунда

In [16]:
%%writefile HW_bots/plus_one_bot.py
import math
from utils import get_score
import random


def plus_one_bot(observation, configuration):
    if observation.step==0:
        return random.randint(0,configuration.signs)
    else:
        return ((observation.lastOpponentAction+1)%configuration.signs)  

Overwriting HW_bots/plus_one_bot.py


Игра со статистом

In [17]:
evaluate(
    "rps", #environment to use - no need to change
    ["HW_bots/plus_one_bot.py", "HW_bots/statistical.py"], #agents to evaluate
    configuration={"episodeSteps": 100} #number of episodes 
)

[[57.0, -57.0]]

### 8. Агент, выдает +2 к предыдущему ходу соперника, вне зависимости от результата раунда

In [18]:
%%writefile HW_bots/plus_two_bot.py
import math
from utils import get_score
import random

def plus_two_bot(observation, configuration):
    if observation.step==0:
        return random.randint(0,configuration.signs)
    else:
        return (observation.lastOpponentAction+2)%configuration.signs

Overwriting HW_bots/plus_two_bot.py


Игра со статистом

In [19]:
evaluate(
    "rps", #environment to use - no need to change
    ["HW_bots/plus_two_bot.py", "HW_bots/statistical.py"], #agents to evaluate
    configuration={"episodeSteps": 100} #number of episodes 
)

[[-99.0, 99.0]]

### 9. Агент, делает то же самое если выиграл, меняет на рандом если не выиграл 

In [20]:
%%writefile HW_bots/win_abride_bot.py
import math
from utils import get_score
import random

lastAction = None

def win_abride_bot(observation, configuration):
    global lastAction 
    if observation.step==0:
        lastAction=random.randint(0,2)
    elif get_score(lastAction, observation.lastOpponentAction) == 1:
        lastAction=random.randint(0,2)
    else: 
        lastAction=lastAction
    
    return lastAction
        
        

Overwriting HW_bots/win_abride_bot.py


Игра со статистом

In [21]:
evaluate(
    "rps", #environment to use - no need to change
    ["HW_bots/win_abride_bot.py", "HW_bots/statistical.py"], #agents to evaluate
    configuration={"episodeSteps": 100} #number of episodes 
)

[[-98.0, 98.0]]

### 10. Агент, делает тоже самое если выиграл, меняет на ход соперника если не выиграл 

In [22]:
%%writefile HW_bots/adjust_to_the_opponent_bot.py
import math
from utils import get_score
import random

lastAction = None

def adjust_to_the_opponent_bot(observation, configuration):
    global lastAction 
    if observation.step==0:
        lastAction=random.randint(0,configuration.signs)
    elif get_score(lastAction, observation.lastOpponentAction) == 1:
        lastAction=observation.lastOpponentAction
    else: 
        lastAction=lastAction
    
    return lastAction

Overwriting HW_bots/adjust_to_the_opponent_bot.py


Игра со статистом

In [23]:
evaluate(
    "rps", #environment to use - no need to change
    ["HW_bots/adjust_to_the_opponent_bot.py", "HW_bots/statistical.py"], #agents to evaluate
    configuration={"episodeSteps": 100} #number of episodes 
)

[[-98.0, 98.0]]

### 11. Агент, выдает +2 если не выиграл, делает тоже самое если выиграл 

In [24]:
%%writefile HW_bots/lost_plus_two_bot.py
import math
from utils import get_score
import random

lastAction = None

def lost_plus_two_bot(observation, configuration):
    global lastAction 
    if observation.step==0:
        lastAction=random.randint(0,2)
    elif get_score(lastAction, observation.lastOpponentAction) != 1:
        lastAction=(lastAction+2)%configuration.signs
    else:
        lastAction
    
    return lastAction

Overwriting HW_bots/lost_plus_two_bot.py


Игра со статистом

In [25]:
evaluate(
    "rps", #environment to use - no need to change
    ["HW_bots/lost_plus_two_bot.py", "HW_bots/statistical.py"], #agents to evaluate
    configuration={"episodeSteps": 100} #number of episodes 
)

[[57.0, -57.0]]

### 12. Агент, если не выиграл рандом. Если выиграл предыдущий ход соперника

In [26]:
%%writefile HW_bots/win_opponent_lost_random_bot.py
import math
from utils import get_score
import random

lastAction = None

def win_opponent_lost_random_bot(observation, configuration):
    global lastAction 
    
    if observation.step==0:
        lastAction=random.randint(0,configuration.signs-1)
    elif get_score(lastAction, observation.lastOpponentAction) != 1:
        lastAction=random.randint(0,configuration.signs-1)
    else:
        
         lastAction=observation.lastOpponentAction 
    
    return lastAction 

Overwriting HW_bots/win_opponent_lost_random_bot.py


Игра со статистом

In [27]:
evaluate(
    "rps", #environment to use - no need to change
    ["HW_bots/win_opponent_lost_random_bot.py", "HW_bots/statistical.py"], #agents to evaluate
    configuration={"episodeSteps": 100} #number of episodes 
)

[[0, 0]]

### 13. Агент: Три хода и рандом. Три раза повторяет свое действие, дальше рандом 

In [28]:
%%writefile HW_bots/two_moves_bot.py

import math
from utils import get_score
import random

Action_history=[]
Action_count=len(Action_history)
lastAction=None


def two_moves_bot(observation, configuration):
    global Action_history
    global Action_count
    global lastAction
    Action_count=len(Action_history)
    if observation.step==0:
        lastAction=random.randint(0,configuration.signs-1)
        Action_history.append(lastAction)
        
    elif Action_count<2:
        lastAction=lastAction
        Action_history.append(lastAction)
        
    else:
        Action_history=[]
        lastAction=random.randint(0,configuration.signs-1)
        Action_history.append(lastAction)
    
    return lastAction
    
        
    
    

Overwriting HW_bots/two_moves_bot.py


In [29]:
evaluate(
    "rps", #environment to use - no need to change
    ["HW_bots/two_moves_bot.py", "HW_bots/statistical.py"], #agents to evaluate
    configuration={"episodeSteps": 100} #number of episodes 
)

[[0, 0]]

## Запускаем турнир и выводим результаты

In [30]:
agents = ['HW_bots/two_moves_bot.py',
         'HW_bots/win_opponent_lost_random_bot.py',
         'HW_bots/lost_plus_two_bot.py',
         'HW_bots/adjust_to_the_opponent_bot.py',
         'HW_bots/win_abride_bot.py',
         'HW_bots/plus_two_bot.py',
         'HW_bots/plus_one_bot.py',
         'HW_bots/sort_bot.py',
         'HW_bots/random_bot.py',
         'HW_bots/cissors_bot.py',
         'HW_bots/paper_bot.py',
         'HW_bots/rock_bot.py',
         'HW_bots/statistical.py']


rank = {agent: 0 for agent in agents}

for times in range(6):
# сколько раундов сыграет каждая пара
          
    for i in range(len(agents)):
        for j in range(i+1, len(agents)):
            e=evaluate(
    "rps", #environment to use - no need to change
    ["HW_bots/two_moves_bot.py", "HW_bots/statistical.py"], #agents to evaluate
    configuration={"episodeSteps": 100,'tieRewardThreshold': 2} #number of episodes 
)
            if e[0][0] > e[0][1]:
                rank[agents[i]] += 1
            elif e[0][0] < e[0][1]:
                rank[agents[j]] += 1
                
                
rank_se = pd.Series(rank, name="Рейтинг турнира по количеству выигранных партий").sort_values(ascending=False).to_frame()
rank_se
            

,Рейтинг турнира по количеству выигранных партий
HW_bots/statistical.py,49
HW_bots/paper_bot.py,45
HW_bots/rock_bot.py,45
HW_bots/cissors_bot.py,39
HW_bots/plus_two_bot.py,33
HW_bots/sort_bot.py,33
HW_bots/random_bot.py,33
HW_bots/plus_one_bot.py,31
HW_bots/win_abride_bot.py,27
HW_bots/lost_plus_two_bot.py,20
